# Baseline Constructive Activation

We want to be able to compare the donor accounts against reasonable baselines of constructive edit activation. This notebook estimates that for the Spanish and Portuguese Wikipedias using an adapted version of the data gathering pipeline for NEWTEA revisited. ([T270786](https://phabricator.wikimedia.org/T270786))

We gather registrations in May and June 2021, because the deployment to Spanish Wikipedia happened in late April of the same year. For those registrations, we gather data on activation and retention, both counting all edits and only constructive edits.

In [1]:
import json
import datetime as dt

from collections import defaultdict

import numpy as np
import pandas as pd

from wmfdata import hive, spark, mariadb

In [2]:
## Start and end timestamps of data gathering, per notes above:
exp_start_ts = dt.datetime(2021, 5, 1, 0, 0, 0)
exp_end_ts = dt.datetime(2021, 7, 1, 0, 0, 0)

## Ordered list of wikis that we'll be gathering data for
## Note that we're excluding euwiki due to their small number of registrations
wikis = ['ptwiki', 'eswiki']

## The MediaWiki history snapshot we use for our data gathering
mwh_snapshot = '2021-07'

## Lists of known users to ignore (e.g. test accounts and experienced users)
known_users = defaultdict(set)
known_users['cswiki'].update([14, 127629, 303170, 342147, 349875, 44133, 100304, 307410, 439792, 444907,
                              454862, 456272, 454003, 454846, 92295, 387915, 398470, 416764, 44751, 132801,
                              137787, 138342, 268033, 275298, 317739, 320225, 328302, 339583, 341191,
                              357559, 392634, 398626, 404765, 420805, 429109, 443890, 448195, 448438,
                              453220, 453628, 453645, 453662, 453663, 453664, 440694, 427497, 272273,
                              458025, 458487, 458049, 59563, 118067, 188859, 191908, 314640, 390445,
                              451069, 459434, 460802, 460885, 79895, 448735, 453176, 467557, 467745,
                              468502, 468583, 468603, 474052, 475184, 475185, 475187, 475188, 294174,
                              402906, 298011])

known_users['kowiki'].update([303170, 342147, 349875, 189097, 362732, 384066, 416362, 38759, 495265,
                              515553, 537326, 566963, 567409, 416360, 414929, 470932, 472019, 485036,
                              532123, 558423, 571587, 575553, 576758, 360703, 561281, 595100, 595105,
                              595610, 596025, 596651, 596652, 596653, 596654, 596655, 596993, 942,
                              13810, 536529])

known_users['viwiki'].update([451842, 628512, 628513, 680081, 680083, 680084, 680085, 680086, 355424,
                              387563, 443216, 682713, 659235, 700934, 705406, 707272, 707303, 707681, 585762])

known_users['arwiki'].update([237660, 272774, 775023, 1175449, 1186377, 1506091, 1515147, 1538902,
                              1568858, 1681813, 1683215, 1699418, 1699419, 1699425, 1740419, 1759328, 1763990])

## Grab the user IDs of known test accounts so they can be added to the exclusion list

def get_known_users(wiki):
    '''
    Get user IDs of known test accounts and return a set of them.
    '''
    
    username_patterns = ["MMiller", "Zilant", "Roan", "KHarlan", "MWang", "SBtest",
                         "Cloud", "Rho2019", "Test"]

    known_user_query = '''
SELECT user_id
FROM user
WHERE user_name LIKE "{name_pattern}%"
    '''
    
    known_users = set()
    
    for u_pattern in username_patterns:
        new_known = mariadb.run(known_user_query.format(
            name_pattern = u_pattern), wiki)
        known_users = known_users | set(new_known['user_id'])

    return(known_users)
        
for wiki in wikis:
    known_users[wiki] = known_users[wiki] | get_known_users(wiki)

## Helper Functions

In [3]:
def make_known_users_sql(kd, wiki_column, user_column):
    '''
    Based on the dictionary `kd` mapping wiki names to sets of user IDs of known users,
    create a SQL expression to exclude users based on the name of the wiki matching `wiki_column`
    and the user ID not matching `user_column`
    '''
    
    wiki_exp = '''({w_column} = '{wiki}' AND {u_column} NOT IN ({id_list}))'''
    
    expressions = list()

    ## Iteratively build the expression for each wiki
    for wiki_name, wiki_users in kd.items():
        expressions.append(wiki_exp.format(
            w_column = wiki_column,
            wiki = wiki_name,
            u_column = user_column,
            id_list = ','.join([str(u) for u in wiki_users])
        ))
    
    ## We then join all the expressions with an OR, and we're done.
    return(' OR '.join(expressions))
    

In [4]:
def make_when_then(wiki_list, wiki_column):
    '''
    Take the ordered list of wiki names and turn it into a string
    of "WHEN wiki_column = '{wiki}' THEN '{k}'" where `k` is the index
    of the wiki in the list, so it can be used for ordering results.
    '''

    whens = list()
    
    for k, wiki in enumerate(wiki_list):
        whens.append(f'WHEN {wiki_column} = "{wiki}" THEN "{k:02}"')
    
    ## Join them with line breaks to create the list
    return('\n'.join(whens))


## User Registrations

We get user registrations for users registered between the start and the end of the experiment, on all wikis, with known test accounts excluded.

In [5]:
## Query to get user registrations on the given wikis for the given dates, ignoring
## known users.

user_registrations_query = '''
SELECT wiki_db, user_id, user_registration_timestamp
FROM wmf.mediawiki_user_history
WHERE snapshot = "{snapshot}"
AND caused_by_event_type = "create" -- account registration
AND created_by_self = true -- no auto-created accounts
AND SIZE(is_bot_by) = 0 -- no bots
AND SIZE(is_bot_by_historical) = 0 -- definitely no bots
AND wiki_db IN ({wiki_list})
AND ({known_user_id_expression})
AND user_registration_timestamp > "{exp_start_timestamp}"
AND user_registration_timestamp < "{exp_end_timestamp}"
'''

In [6]:
## NOTE: if you use growth.utils.hive_format to format the timestamps, then you miss the first day of data
## as the "T" in the Hive format isn't in the timestamps in mediawiki_history. And you probably get
## an extra day of data at the end.

user_registrations = spark.run(
    user_registrations_query.format(
        snapshot = mwh_snapshot,
        wiki_list = ','.join(['"{}"'.format(w) for w in wikis]),
        known_user_id_expression = make_known_users_sql(known_users, 'wiki_db', 'user_id'),
        exp_start_timestamp = exp_start_ts.strftime('%Y-%m-%d %H:%M:%S'),
        exp_end_timestamp = exp_end_ts.strftime('%Y-%m-%d %H:%M:%S')
    )
)

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [7]:
## verify first registration in the dataset
user_registrations['user_registration_timestamp'].min()

'2021-05-01 00:00:04.0'

In [8]:
## verify last registration in the dataset
user_registrations['user_registration_timestamp'].max()

'2021-06-30 23:59:35.0'

Those timestamps fit our requirements.

## Get mobile/desktop and API data

This enables us to identify differences between mobile/desktop registrations. We also filter out API registrations as those are almost always mobile apps.

In [9]:
## Get all self-created registrations from ServerSideAccountCreation

ssac_reg_query = '''
SELECT wiki AS wiki_db, event.userid AS user_id,
       CAST(event.displaymobile AS INT) AS reg_on_mobile,
       event.isapi
FROM event_sanitized.serversideaccountcreation
WHERE year = 2021
AND month BETWEEN 4 AND 7
AND wiki IN ({wiki_list})
AND event.isselfmade = true
'''

In [10]:
ssac_reg_data = spark.run(ssac_reg_query.format(
    wiki_list = ','.join(['"{}"'.format(w) for w in wikis])
))

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [11]:
## What's the number of registrations?

user_registrations.groupby('wiki_db').agg({'user_id' : 'size'})

,user_id
wiki_db,
eswiki,41242
ptwiki,21467


In [12]:
ssac_reg_data.groupby('wiki_db').agg({'user_id' : 'size'})

,user_id
wiki_db,
eswiki,78752
ptwiki,43832


In [13]:
## Join with SSAC data

all_users = user_registrations.merge(ssac_reg_data, how = 'left',
                                    on = ['wiki_db', 'user_id'])

In [14]:
## Did we lose anyone?

all_users.loc[all_users['reg_on_mobile'].isna()].groupby('wiki_db').agg({'user_id' : 'size'})

,user_id
wiki_db,
eswiki,1


There's only a single user that we don't appear to have registration data for, so that means that we're good to go. Yay!

We remove users that have `isapi = True`, because those are most likely app registrations.

In [15]:
all_users = all_users.loc[~(all_users['isapi'] == True)]

In [16]:
## Drop the isapi columns

all_users = all_users.drop(columns = ['isapi'])

## Get treatment/control assignments

This involves two operations.

1. Get all users who have the Homepage turned on in their preferences.
2. Get the variant setting for all users.

Users who don't have the Homepage turned on are candidates for the control group, and likewise for the experiment group. Secondary, users are randomly assigned to variants, which we'll use later in the analysis.

In [17]:
def get_prop_settings(wiki, prop, col_name, cast_type, users=None):
    '''
    Query and return a `pandas.DataFrame` with columns `wiki` and `user_id` of all users who have
    the given property turned on in their preferences for that given wiki.
    
    :param wiki: database code of the wiki we're querying
    :type wiki: str
    
    :param prop: the user preference we're querying for
    :type prop: str
    
    :param col_name: name that the column with preference value should have in the
                     resulting DataFrame (e.g. "is_treatment")
    :type col_name: str
    
    :param cast_type: SQL type to cast the property to (in the database this is a BLOB)
    :type cast_type: str
    
    :param users: user IDs of the users we are interested in. This is optional.
    :type users: list
    '''
    
    prop_query = '''
    SELECT "{wiki}" AS wiki, up_user AS user_id,
           CAST(up_value AS {cast_type}) AS {col_name}
    FROM user_properties
    WHERE up_property = "{prop}"
    '''.format(wiki = wiki, prop = prop, cast_type = cast_type, col_name = col_name)
    
    if users is not None:
        prop_query += '''
        AND up_user IN ({})
        '''.format(','.join([str(uid) for uid in users]))
        
    return(mariadb.run(prop_query, wiki))

In [18]:
## Get treatment/control assignments from the MW databases

hp_prefs = pd.concat(
    [get_prop_settings(wiki,
                       'growthexperiments-homepage-enable',
                       'hp_enabled',
                       'UNSIGNED INTEGER') for wiki in wikis]
)

In [19]:
all_users = all_users.merge(hp_prefs, how = 'left',
                            left_on = ['wiki_db', 'user_id'],
                            right_on = ['wiki', 'user_id']).fillna(0)

In [20]:
all_users_agg = (all_users.groupby(['wiki_db', 'hp_enabled'])
                 .agg({'user_id': 'count'})
                 .rename(columns = {'user_id' : 'n_users'}))
all_users_agg['perc'] = (100.0 * all_users_agg['n_users'] / 
                        all_users_agg.groupby('wiki_db')['n_users'].transform('sum'))
all_users_agg

n_users       perc
wiki_db hp_enabled                    
eswiki  0.0            8996  23.547889
        1.0           29207  76.452111
ptwiki  0.0            4759  23.658961
        1.0           15356  76.341039

This is just routine, the random assignment hasn't failed in any of our experiments so far. From the percentages above it's clear that it continues to work, but the proportions are off by about 3%. We're expecting an 80/20 split. I'll investigate that.

Said investigation found that [T285996](https://phabricator.wikimedia.org/T285996) caused the randomization to be switched around. Registrations in June prior to that bug get the right proportions assigned.

In [21]:
## Get variant settings from the MW database

variant_prefs = pd.concat(
    [get_prop_settings(wiki,
                       'growthexperiments-homepage-variant',
                       'hp_variant',
                       'CHAR CHARACTER SET utf8') for wiki in wikis]
)

In [22]:
all_users = all_users.merge(variant_prefs, how = 'left',
                           left_on = ['wiki_db', 'user_id'],
                           right_on = ['wiki', 'user_id']).fillna(0)

In [23]:
all_users.groupby(['hp_enabled', 'hp_variant']).agg({'user_id': 'count'})

user_id
hp_enabled hp_variant         
0.0        0             13736
           control          19
1.0        0                33
           control       42138
           null           2392

We currently don't have any variant tests running and Add a Link was deployed in May. I'm thinking that's why everyone has the "control" variant assignment here. Looks like 19 users turned the Homepage off, and 33 turned it on. We'll exclude those from the analysis and keep everyone else.

In [ ]:
## Control group users cannot have a variant assigned
all_users.loc[(all_users['hp_enabled'] == 0) &
              (all_users['hp_variant'] == 'control')]

In [ ]:
## Homepage group users have to have a variant assigned
all_users.loc[(all_users['hp_enabled'] == 1) &
              (all_users['hp_variant'] == 0)]

In [26]:
all_users_valid = all_users.loc[
    ((all_users['hp_enabled'] == 0) &
     (all_users['hp_variant'] != 'control')) |
    ((all_users['hp_enabled'] == 1) &
     (all_users['hp_variant'] != 0))
]

In [27]:
len(all_users)

58318

In [28]:
len(all_users_valid)

58266

We've removed 52 users, so those numbers make sense.

In [29]:
all_users = all_users_valid.copy() # if we don't copy, we'll get warnings later.

## Edit Data

We create a temporary Spark dataframe to store our user data, then join that with MediaWiki history to aggregate edit data.

In [30]:
## First we drop the `wiki_x` and `wiki_y` columns.
all_users.drop(['wiki_x', 'wiki_y'], axis = 'columns', inplace = True)

In [ ]:
all_users.head()

In [32]:
## Turn the two 0/1 columns into ints, as they should be, and force hp_variant to string.
all_users['reg_on_mobile'] = all_users['reg_on_mobile'].astype(int)
all_users['hp_enabled'] = all_users['hp_enabled'].astype(int)
all_users['hp_variant'] = all_users['hp_variant'].astype(str)

In [33]:
spark_session = spark.get_session()
all_users_sdf = spark_session.createDataFrame(all_users)
all_users_sdf.createGlobalTempView("growth_all_users_temp")

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [34]:
edit_data_query = '''
WITH edits AS (
    SELECT wiki_db, event_user_id AS user_id,
    -- ns 0 & 1 edits on the first day
    SUM(IF(page_namespace IN (0, 1)
        AND unix_timestamp(event_timestamp) - unix_timestamp(event_user_creation_timestamp) < 86400, 1, 0))
        AS num_article_edits_24hrs,
    -- ns 0 & 1 edits on the first day that were reverted
    SUM(IF(page_namespace IN (0, 1) AND revision_is_identity_reverted = true AND revision_seconds_to_identity_revert < 60*60*48
        AND unix_timestamp(event_timestamp) - unix_timestamp(event_user_creation_timestamp) < 86400, 1, 0))
        AS num_article_reverts_24hrs,
    --  other namespace edits on the first day
    SUM(IF(page_namespace NOT IN (0, 1)
        AND unix_timestamp(event_timestamp) - unix_timestamp(event_user_creation_timestamp) < 86400, 1, 0))
        AS num_other_edits_24hrs,
    -- other namespace reverts on the first day
    SUM(IF(page_namespace NOT IN (0, 1) AND revision_is_identity_reverted = true AND revision_seconds_to_identity_revert < 60*60*48
        AND unix_timestamp(event_timestamp) - unix_timestamp(event_user_creation_timestamp) < 86400, 1, 0))
        AS num_other_reverts_24hrs,
    -- ns 0 & 1 edits on days 1–15
    SUM(IF(page_namespace IN (0, 1)
        AND unix_timestamp(event_timestamp) - unix_timestamp(event_user_creation_timestamp) BETWEEN 86400 AND 15*86400, 1, 0))
        AS num_article_edits_2w,
    -- ns 0 & 1 edits on days 1–15 that were reverted
    SUM(IF(page_namespace IN (0, 1) AND revision_is_identity_reverted = true AND revision_seconds_to_identity_revert < 60*60*48
        AND unix_timestamp(event_timestamp) - unix_timestamp(event_user_creation_timestamp) BETWEEN 86400 AND 15*86400, 1, 0))
        AS num_article_reverts_2w,
    -- other namespace edits on days 1–15
    SUM(IF(page_namespace NOT IN (0, 1)
        AND unix_timestamp(event_timestamp) - unix_timestamp(event_user_creation_timestamp) BETWEEN 86400 AND 15*86400, 1, 0))
        AS num_other_edits_2w,
    -- other namespace reverts on days 1–15
    SUM(IF(page_namespace NOT IN (0, 1) AND revision_is_identity_reverted = true AND revision_seconds_to_identity_revert < 60*60*48
        AND unix_timestamp(event_timestamp) - unix_timestamp(event_user_creation_timestamp) BETWEEN 86400 AND 15*86400, 1, 0))
        AS num_other_reverts_2w
    FROM wmf.mediawiki_history
    WHERE snapshot = "{snapshot}"
    AND event_entity = "revision"
    AND event_type = "create"
    AND wiki_db IN ({wiki_list})
    AND event_timestamp > "{start_date}"
    AND event_timestamp < "{end_date}"
    GROUP BY wiki_db, event_user_id
),
users AS (
    SELECT
        wiki_db,
        user_id,
        user_registration_timestamp,
        reg_on_mobile,
        hp_enabled,
        hp_variant
    FROM global_temp.growth_all_users_temp
)
SELECT
    users.wiki_db,
    users.user_id,
    users.user_registration_timestamp,
    users.reg_on_mobile,
    users.hp_enabled,
    users.hp_variant,
    COALESCE(num_article_edits_24hrs, 0) AS num_article_edits_24hrs,
    COALESCE(num_article_reverts_24hrs, 0) AS num_article_reverts_24hrs,
    COALESCE(num_other_edits_24hrs, 0) AS num_other_edits_24hrs,
    COALESCE(num_other_reverts_24hrs, 0) AS num_other_reverts_24hrs,
    COALESCE(num_article_edits_2w, 0) AS num_article_edits_2w,
    COALESCE(num_article_reverts_2w, 0) AS num_article_reverts_2w,
    COALESCE(num_other_edits_2w, 0) AS num_other_edits_2w,
    COALESCE(num_other_reverts_2w, 0) AS num_other_reverts_2w
FROM users
LEFT JOIN edits
ON users.wiki_db = edits.wiki_db
AND users.user_id = edits.user_id
'''

In [35]:
## We set the start date to the experiment start date,
## and the end date to the experiment end date + 15 days
## to give users who registered within 15 days of the last
## date the same amount of time to edit as everyone else.

all_users_edit_data = spark.run(
    edit_data_query.format(
        snapshot = mwh_snapshot,
        wiki_list = ','.join(['"{}"'.format(w) for w in wikis]),
        start_date = exp_start_ts.date().isoformat(),
        end_date = (exp_end_ts.date() + dt.timedelta(days = 15)).isoformat()
    )
)

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [ ]:
all_users_edit_data.head()

## Define Activation

We'll use the same definition of activation as we've used previously: editing within 24 hours of registration. We calculate this both across all namespaces, and specifically for the Article (and Article talk) namespace.

In [37]:
all_users_edit_data['is_activated'] = False
all_users_edit_data.loc[
    (all_users_edit_data['num_article_edits_24hrs'] > 0) |
    (all_users_edit_data['num_other_edits_24hrs'] > 0), 'is_activated'
] = True

In [38]:
all_users_edit_data['is_activated_article'] = False
all_users_edit_data.loc[
    (all_users_edit_data['num_article_edits_24hrs'] > 0), 'is_activated_article'
] = True

In [39]:
all_users_edit_data['is_constr_activated'] = False
all_users_edit_data.loc[
    (all_users_edit_data['num_article_edits_24hrs'] - all_users_edit_data['num_article_reverts_24hrs'] > 0) |
    (all_users_edit_data['num_other_edits_24hrs'] - all_users_edit_data['num_other_reverts_24hrs'] > 0),
    'is_constr_activated'] = True

In [40]:
all_users_edit_data['is_constr_activated_article'] = False
all_users_edit_data.loc[
    (all_users_edit_data['num_article_edits_24hrs'] - all_users_edit_data['num_article_reverts_24hrs'] > 0),
    'is_constr_activated_article'] = True

In [ ]:
all_users_edit_data.loc[all_users_edit_data['is_activated'] == True].head()

## Define Registration Month

We also want to split by month because we're curious about trends across time.

Since the registration timestamp is an ISO 8601 timestamp, we grab the first 7 characters of it ("YYYY-MM").

In [42]:
all_users_edit_data['reg_month'] = all_users_edit_data['user_registration_timestamp'].apply(
    lambda x: x[:7]
)

## Aggregate

We can now select users in the Homepage group, aggregate across wiki and activation, and calculate the percentages.

In [48]:
growth_activation_baseline_agg = (
    all_users_edit_data.loc[all_users_edit_data['hp_enabled'] == 1]
    .groupby('is_constr_activated')
    .agg({'user_id' : 'count'})
    .rename(columns = {'user_id' : 'num_users'}))
growth_activation_baseline_agg['perc'] = (
    100 * growth_activation_baseline_agg['num_users'] /
    growth_activation_baseline_agg['num_users'].sum())
growth_activation_baseline_agg.round(1)

,num_users,perc
is_constr_activated,,
False,32507,73.0
True,12023,27.0


In [46]:
growth_activation_baseline_agg_by_wiki = (
    all_users_edit_data.loc[all_users_edit_data['hp_enabled'] == 1]
    .groupby(['wiki_db', 'is_constr_activated'])
    .agg({'user_id' : 'count'})
    .rename(columns = {'user_id' : 'num_users'}))
growth_activation_baseline_agg_by_wiki['perc'] = (
    100 * growth_activation_baseline_agg_by_wiki['num_users'] /
    growth_activation_baseline_agg_by_wiki.groupby(['wiki_db'])['num_users'].transform('sum'))
growth_activation_baseline_agg_by_wiki.round(1)

num_users  perc
wiki_db is_constr_activated                 
eswiki  False                    21986  75.3
        True                      7203  24.7
ptwiki  False                    10521  68.6
        True                      4820  31.4

By month of registration:

In [44]:
growth_activation_baseline_monthly_agg = (
    all_users_edit_data.loc[all_users_edit_data['hp_enabled'] == 1]
    .groupby(['wiki_db', 'reg_month', 'is_constr_activated'])
    .agg({'user_id' : 'count'})
    .rename(columns = {'user_id' : 'num_users'}))
growth_activation_baseline_monthly_agg['perc'] = (
    100 * growth_activation_baseline_monthly_agg['num_users'] /
    growth_activation_baseline_monthly_agg.groupby(['wiki_db', 'reg_month'])['num_users'].transform('sum'))
growth_activation_baseline_monthly_agg.round(1)

num_users  perc
wiki_db reg_month is_constr_activated                 
eswiki  2021-05   False                    12887  76.0
                  True                      4078  24.0
        2021-06   False                     9099  74.4
                  True                      3125  25.6
ptwiki  2021-05   False                     6001  69.1
                  True                      2680  30.9
        2021-06   False                     4520  67.9
                  True                      2140  32.1

I noticed the large drop in number of registrations between May and June for both wikis. Comparing it with the number of registrations from Wikistats, the drop is consistent. End of school year and vacation impacting the numbers again? I suspect so.